# Capítulo 16: Natural language processing with RNNs and attention (página 525)

Una aproximación común al realizar NLP es utilizar redes neuronales recurrentes (RNN)
En este primer ejercicio haremos unpredictor de caracteres (qué caracter iría después de otro probablemente)
RNN stateless --> Aprende de posiciones aleatorias de texto en cada iteración sin información alguna del resto del texto.
RNN stateful --> Que preserva un "estado" oculto a través de las iteraciones de entrenamiento y continúa leyendo donde lo dejó previamente.

# Importando nuestros paquetes

In [1]:
from tensorflow import keras
import tensorflow as tf
import numpy as np

# Creando el dataset de entrenamiento

Primero, descargamos el dataset del trabajo de Shakespeare's utilizando Keras

In [2]:
shakespeare_url = "https://homl.info/shakespeare" # URL recortada
filepath = keras.utils.get_file('Shakespeare.txt', shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

Ahora que ya tenemos el texto cargado en memoria, necesitamos tokenizarlo, esto significa que daremos un valor numérico a cada caracter único presente en nuestro set de datos. Para ello, utilizaremos Tokenizer de Keras.

In [3]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_text)

Establecemos `char_level=True` para obtener los tokens a nivel de caracter y no de palabra, como lo hace ya por defecto la función.
Nota que tokenizer convierte el texto a minúsculas por defecto, pero puedes utilizar el argumento `lower=True` para evitar este comportamiento.

Ahora, el tokenizador puede codificar un enunciado (o una lista de enunciados) a una lista de IDs de caracteres y viceversa, y entonces nos puede decir que tantos caracteres distintos hay y el total de caracteres distintos en el texto.

## Ejemplo del uso de tokenizer:

In [4]:
tokenizer.texts_to_sequences(['First'])

[[20, 6, 9, 8, 3]]

In [5]:
tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]])

['f i r s t']

In [6]:
max_id = len(tokenizer.word_index) # total de caracteres distintos
max_id

39

In [7]:
dataset_size = tokenizer.document_count # total de caracteres en el documento
dataset_size

1115394

Vamos a codificar el texto completo así, cada caracter es representado por si propio ID (restaremos 1 a cada ID para obtener IDs del 0 al 38 en lugar del 1 al 39):

In [8]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1
encoded

array([19,  5,  8, ..., 20, 26, 10])

## Dividamos nuestro dataset en un set de entrenamiento y otro de prueba

Es muy importante evitar cualquier traslape entre nuestro set de entrenamiento, el set de validación y el set de prueba. Por ejemplo, podemos tomar el primer 90% de el texto para el set de entrenamiento, el siguiente 5% para el set de validación y el resto para el set de prueba.

Entonces, tomamos el 90% del dataset como set de entrenamiento y el resto para prueba y validación, también creamos una instancia de tf.data.Dataset que retornará cada caracter, uno por uno de este set:

In [9]:
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

**Espacio para explicar el uso de encoded**

Ahora que tenemos nuestro set de entrenamiento, este consiste en una única secuencia de más de un millón de caracteres; no podemos entrenar nuestra red neuronal directamente sobre este set: nuestra RNN sería equivalente a un red profunda con más de un millón de capas, y tendríamos una sola instancia (pero bastante larga) sobre la cual entrenar (nuestras entradas).

En lugar de esto, utilizaremos el método `window()` del dataset, que convertirá esta larga secuencia de caracteres en pequeñas ventanas (muestras) de texto. Cada instancia en el dataset será una subcadena del texto completo, y la RNN será entrenada en cada ocasión únicamente con los caracteres de la muestra. Esto es llamado ***truncated backprogation through time***. Utilicemos el método `window()` sobre el dataset:

In [10]:
n_steps = 100
window_length = n_steps + 1 # objetivo = entrada desplazada 1 carácter adelante
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

***Nota: Puedes intentar ir cambiando n_steps: es más fácil entrenar RNNs con secuencias cortas de caracteres pero, por supuesto, la RNN no será capaz de aprender ningún patrón de una longitud más grande que n_steps, así que procura no hacerlo demasiado corto***

Por defecto, el método `window()` genera ventanas que no se traslapan, pero para obtener el set de entrenamiento más largo posible utilizamos `shift=1`, de esta manera, la primera ventana contendrá los caracteres del 0 al 100 (100 porque es el número al que hemos igualado n_steps), la segunda contendrá los caracteres del 1 al 101, y así sucesivamente. Para asegurarnos que todas las ventanas sean de exactamente 101 caracteres (lo cual nos permitirá crear batches sin tener que agregar algún tipo de relleno), establecemos `drop_remainder=True`(de otra manera, las últimas 100 ventanas contendrán 100 caracteres, 99 caracteres, y así sucesivamente hasta llegar a 1 caracter).

# Convirtiendo datasets anidados a tensores

EL método `window()` generará un dataset que contiene ventanas, donde cada una de ellas es representada igualmente como un dataset. Es un dataset anidado, análogo a una lista de listas. Esto es útil cuando queremos transformar las ventanas utilizando los métodos de dataset (por ejemplo, para intercambiarlos o muestrearlos). Sin embargo, no podemos utilizar un dataset anidado directamente, dado que nuestro modelo espera tensores, no datasets. Por ello, es necesario llamar al método `flat_map()`, este convierte un *dataset anidado* en un *dataset plano*.

In [11]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

Note que utilizamos la variable `windows_length` dado que todas las ventanas tienen la misma longitud: dado que todas las ventanas tienen exactamente la misma longitud, obtendremos un único tensor por cada una de ellas. Ahora, el dataset contendrá ventanas consecutivas de 101 caracteres cada una.

Mientras que el descenso de gradiente funciona mejor cuando las instancias en el set de entrenamiento son independientes y distribuidas de manera uniforme, necesitamos reordenar esas ventanas. Solo así podremos generar muestras de esas ventanas y separar las entradas (los primeros 100 caracteres) del objetivo (el último caracter).

In [12]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

Las variables categóricas suelen ser codificadas, generalmente como vectores one-hot o como embeddings. Aquí, codificaremos cada caracter utilizando un vector one-hot porque tenemos realmente pocos caracteres únicos (solo 39)

In [13]:
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch)
)

# Finalmente, necesitamos únicamente añadir precarga:
dataset = dataset.prefetch(1)

# Construyendo y entrenando el modelo Char-NN

Para predecir el siguiente caracter basado en los 100 anteriores, podemos utilizar una Red Neuronal Recurrente (RNN) con dos capas GRU de 128 unidades cada una y 20% de dropout para ambas entradas (dropout) y los estados ocultos (recurrent_dropout). La capa de salida es una capa Dense distribuida-temporalmente. Esta vez, la capa debe tener 39 unidades (max_id) porque hay 39 caracteres distintos en el texto, y deseamos obtener la probabilidad para cada posible caracter (una vez, por cada paso).

In [18]:
model = keras.models.Sequential([keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id], dropout=0.2, recurrent_dropout=0.2),
    keras.layers.GRU(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation="softmax"))
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=20)

NotImplementedError: Cannot convert a symbolic Tensor (gru_8/strided_slice:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported

# Utilizando el modelo Char-NN

Ahora que ya tenemos el modelo que puede predecir el siguiente caracter en el texto escrito por Shakespear. Para alimentarlo con algo de texto, primero necesitamos preprocesarlo como hicimos anteriormente, así que crearemos una función para ello:

In [ ]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

Probemos nuestro modelo para predecir el siguiente caracter de una frase típica:

In [ ]:
X_new = preprocess(["How are yo"])
Y_pred = model.predict_classes(X_new)
tokenizer.sequences_to_texts(Y_pred + 1)[0][1]